In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Library

In [2]:
! pip install -qU \
  "langchain-core>=0.3" \
  "langchain-community>=0.3" \
  "langchain-classic>=0.3" \
  "langchain-openai>=0.2" \
  "langchain-chroma" \
  sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB

In [3]:
! pip install -q underthesea
! pip install -qU  rank_bm25
! pip -q install -qU pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.39.1 which is incompatible.
google-adk 1.21.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.
gradio 5.50.0 requires pydantic<=2.12.3,>=2.0, but you have pydantic 2.12.5 which is incompatible.


In [4]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_classic.retrievers.document_compressors import CrossEncoderReranker
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_core.vectorstores.utils import maximal_marginal_relevance

from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from sentence_transformers import SentenceTransformer
from langchain_core.embeddings import Embeddings
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from pathlib import Path
from typing import Optional, Dict, Any, List, Tuple
from collections import defaultdict
from underthesea import word_tokenize
from pydantic import BaseModel, Field
from google.colab import userdata
from google.colab import files
from PIL import Image
from IPython.display import display
from openai import OpenAI
from dataclasses import dataclass, field

import time, random
import json
import re
import os
import numpy as np
import base64
import gradio as gr
import io
import math

# 2. Load vectorDB

In [5]:
TEXT_EMB_MODEL = "AITeamVN/Vietnamese_Embedding"

text_embedder = HuggingFaceEmbeddings(
    model_name=TEXT_EMB_MODEL,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

/tmp/ipython-input-851266541.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  text_embedder = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [6]:
class ClipImageEmbeddings(Embeddings):
    def __init__(self, model_name: str = "sentence-transformers/clip-ViT-B-32", device: str = "cuda"):
        self.model = SentenceTransformer(model_name, device=device)

    def _encode_path(self, image_path: str) -> List[float]:
        img = Image.open(image_path).convert("RGB")
        vec = self.model.encode(img, convert_to_numpy=True, normalize_embeddings=True)
        return vec.tolist()

    # dùng khi index (texts chính là list path ảnh)
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._encode_path(p) for p in texts]

    # dùng khi similarity_search(path)
    def embed_query(self, text: str) -> List[float]:
        return self._encode_path(text)

In [7]:
chroma_law = Chroma(
    persist_directory=str("/content/drive/Othercomputers/My Laptop/multimodal-rag/chroma_text_law"),
    embedding_function=text_embedder,
)

/tmp/ipython-input-3020774838.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  chroma_law = Chroma(


In [8]:
chroma_law2 = Chroma(
    persist_directory=str("/content/drive/Othercomputers/My Laptop/multimodal-rag/chroma_text_law2"),
    embedding_function=text_embedder,
)

In [9]:
chroma_qa600 = Chroma(
    persist_directory=str("/content/drive/Othercomputers/My Laptop/multimodal-rag/chroma_qa_600"),
    embedding_function=text_embedder,
)

In [10]:
IMG_EMB_MODEL = "sentence-transformers/clip-ViT-B-32"
img_embedder = ClipImageEmbeddings(model_name=IMG_EMB_MODEL, device="cuda")

chroma_img600 = Chroma(
    persist_directory="/content/drive/Othercomputers/My Laptop/multimodal-rag/chroma_img_600",
    embedding_function=img_embedder,
)

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

0_CLIPModel/model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

0_CLIPModel/pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [11]:
def embed_uploaded_bytes_to_vec(img_bytes: bytes) -> list[float]:
    pil = Image.open(io.BytesIO(img_bytes)).convert("RGB")
    vec = img_embedder.model.encode(pil, convert_to_numpy=True, normalize_embeddings=True)
    return vec.tolist()

# 3. Hybrid Search

In [12]:
def load_chunks_flexible(path: str, default_source: str | None = None):
    p = Path(path)
    text = p.read_text(encoding="utf-8").replace("\r\n", "\n")  # ăn cả CRLF/LF

    # Bắt từng CHUNK block
    chunk_pat = re.compile(
        r"^===== CHUNK (?P<id>\d+) =====\s*\n(?P<body>.*?)(?=^===== CHUNK|\Z)",
        re.S | re.M,
    )

    docs = []
    for m in chunk_pat.finditer(text):
        chunk_id = int(m.group("id"))
        body = m.group("body").strip()

        # Thử parse metadata nếu có Source/Page ở đầu body
        meta_pat = re.compile(
            r"^Source:\s*(?P<source>.*?)\s*\n"
            r"Page:\s*(?P<page>.*?)\s*\n\s*\n"
            r"(?P<content>.*)$",
            re.S,
        )
        mm = meta_pat.match(body)

        if mm:
            source = mm.group("source").strip()
            page_raw = mm.group("page").strip()
            content = mm.group("content").strip()
            page = int(page_raw) if page_raw.isdigit() else None
        else:
            source = default_source or p.name
            page = None
            content = body

        if not content:
            continue

        docs.append(Document(
            page_content=content,
            metadata={"chunk_id": chunk_id, "source": source, "page": page},
        ))

    return docs


# Dùng:
chunk_text_law = load_chunks_flexible("/content/drive/Othercomputers/My Laptop/multimodal-rag/chunks/chunk_text_law.txt")
chunk_text_law2 = load_chunks_flexible("/content/drive/Othercomputers/My Laptop/multimodal-rag/chunks/chunk_text_law2.txt")
chunk_qa = load_chunks_flexible("/content/drive/Othercomputers/My Laptop/multimodal-rag/chunks/qa_chunks.txt")

In [13]:
retriever_law  = chroma_law.as_retriever(search_kwargs={"k": 4})
retriever_law2 = chroma_law2.as_retriever(search_kwargs={"k": 4})
retriever_qa   = chroma_qa600.as_retriever(search_kwargs={"k": 4})

In [14]:
all_docs = chunk_text_law + chunk_text_law2 + chunk_qa

In [15]:
bm25 = BM25Retriever.from_documents(
    all_docs,
    k=8,
    preprocess_func=lambda s: word_tokenize(s, format="text").split(),
)

hybrid_retriever = EnsembleRetriever(
    retrievers=[retriever_law, retriever_law2, retriever_qa, bm25],
)

In [16]:
def r_invoke(r, q: str):
    if hasattr(r, "invoke"):
        return r.invoke(q)  # ✅ chuẩn mới
    if hasattr(r, "get_relevant_documents"):
        return r.get_relevant_documents(q)  # fallback cho bản cũ
    raise AttributeError(f"{type(r).__name__} has neither invoke nor get_relevant_documents")


def bm25_by_qid(qid: str, k: int = 8):
    bm25.k = k
    return bm25.invoke(f"id: {qid}")   # ✅ dùng invoke


# 4. Multi-queries

In [17]:
raw = userdata.get("RAG")
key = raw.strip()
os.environ["OPENAI_API_KEY"] = key

In [18]:
client = OpenAI()

# --- Request counter (OpenAI) ---
REQ_COUNT = 0

def openai_call_once(*, model: str, input, text_format=None, **kwargs):
    """Wrapper để đếm số lần gọi OpenAI API (mỗi call = 1 request)."""
    global REQ_COUNT
    REQ_COUNT += 1
    return client.responses.parse(
        model=model,
        input=input,
        text_format=text_format,
        **kwargs,
    )

class MultiQueriesOut(BaseModel):
    queries: List[str] = Field(description="Danh sách truy vấn")
    hyde_doc: str = Field(description="Đoạn HyDE để embed cho vector retrieval")

def generate_multi_queries_openai(
    question: str,
    k: int = 3,
    model: str = "gpt-4o-mini",
) -> Tuple[List[str], str]:
    """
    1 request duy nhất.
    - Không retry
    - Không fallback model
    - Nếu parse lỗi => fallback về [question] (KHÔNG phát sinh request thêm)
    """
    prompt = f"""
Bạn là bộ tạo truy vấn cho hệ thống RAG tiếng Việt chuyên về:
- Luật/Quy định giao thông đường bộ Việt Nam, nghị định xử phạt
- Bộ câu hỏi ôn thi giấy phép lái xe (dạng trắc nghiệm + đáp án)

Mục tiêu: tạo truy vấn giúp retriever (BM25 + Vector) tìm đúng đoạn trong corpus.

NHIỆM VỤ (bắt buộc):
1) Tạo đúng {k} truy vấn tìm kiếm khác nhau cho câu hỏi người dùng.
   - Query #1 (BM25 mạnh): ngắn, đúng từ khóa, ưu tiên thuật ngữ “chuẩn đề thi/chuẩn luật”.
   - Query #2 (Vector mạnh): diễn giải/đồng nghĩa, nhưng giữ nguyên thực thể quan trọng (hạng GPLX, số chỗ, điều kiện, hành vi...).
   - Query #3 (văn phong pháp lý/đề thi): đưa vào các cụm “được điều khiển loại xe nào”, “không được phép”, “tốc độ tối đa”, “phải giảm tốc độ hoặc dừng lại”, “thiết bị an toàn cho trẻ em”, “hành vi bị nghiêm cấm”, “có hiệu lực thi hành”, v.v.
   - Nếu {k} > 3: tạo thêm query biến tấu theo hướng (a) thêm từ khóa đồng nghĩa, (b) thêm cụm “quy định”, “đáp án đúng”, “câu hỏi”, “chương”, (c) thêm tên văn bản xử phạt (NĐ 100/2019, sửa đổi 123/2021) CHỈ KHI câu hỏi hỏi mức phạt/chế tài.

2) Tạo 1 đoạn HyDE (hypothetical document) để embed cho vector retrieval:
   - 6–10 câu, viết như “đoạn giải thích/ghi chú ôn thi” hoặc “trích yếu hướng dẫn tra cứu”.
   - Nêu rõ: khái niệm/hành vi/tình huống và tiêu chí phân loại (ví dụ: số chỗ, điều kiện đường, khu vực đông dân cư...).
   - TUYỆT ĐỐI KHÔNG bịa số Điều/Khoản/mức phạt cụ thể nếu không chắc.
   - Nếu câu hỏi có con số quan trọng (ví dụ: “trên 16 chỗ đến 29 chỗ”, “50 km/h”) thì PHẢI giữ nguyên.

RÀNG BUỘC FORMAT:
- Chỉ trả về JSON đúng schema sau, KHÔNG thêm chữ nào khác:
{{
  "queries": ["...","...","..."],
  "hyde_doc": "..."
}}

QUY TẮC CHẤT LƯỢNG:
- Mỗi query ~8–18 từ là tốt (Query #3 có thể dài hơn một chút).
- Ưu tiên từ/cụm đúng văn phong đề thi: “đáp án đúng”, “các phương án”, “trường hợp nào”, “loại xe nào”, “được phép/không được phép”.
- Không thêm thông tin không liên quan; không “bịa luật”.

========================
VÍ DỤ THAM KHẢO PHONG CÁCH (few-shot)
Chỉ học cách viết truy vấn + HyDE, không copy y nguyên.

Ví dụ 1
Câu hỏi: "Xe ưu tiên là gì và có những loại nào trong quy định giao thông đường bộ?"
JSON:
{{
  "queries": [
    "Xe ưu tiên gồm những loại nào quy định giao thông đường bộ",
    "định nghĩa xe ưu tiên và các loại xe được quyền ưu tiên khi tham gia giao thông",
    "xe ưu tiên xe chữa cháy xe cứu thương xe quân sự công an kiểm sát CSGT dẫn đường"
  ],
  "hyde_doc": "Câu hỏi yêu cầu xác định khái niệm 'xe ưu tiên' và liệt kê các loại xe thuộc nhóm này theo quy định giao thông đường bộ. Khi tra cứu, cần tìm mục quy định về xe ưu tiên và danh mục phương tiện làm nhiệm vụ khẩn cấp. Nội dung thường nêu rõ các nhóm như xe chữa cháy, xe cứu thương, xe của lực lượng vũ trang khi làm nhiệm vụ và đoàn xe có CSGT dẫn đường. Trọng tâm là danh sách các loại xe và điều kiện 'đang làm nhiệm vụ'. Nếu có phần tín hiệu ưu tiên, có thể dùng để đối chiếu nghĩa vụ nhường đường. Nên lấy nguyên câu/đoạn liệt kê để trả lời."
}}


Ví dụ 2
Câu hỏi: "Luật số 30/2023/QH15 có hiệu lực từ khi nào và có ảnh hưởng gì đến các luật khác không?"
JSON:
{{
  "queries": [
    "Luật 30/2023/QH15 có hiệu lực thi hành từ ngày nào",
    "thời điểm có hiệu lực Luật 30/2023/QH15 và quy định hết hiệu lực văn bản liên quan",
    "Luật 30/2023/QH15 điều khoản thi hành hiệu lực sửa đổi bổ sung hết hiệu lực"
  ],
  "hyde_doc": "Câu hỏi tập trung vào điều khoản thi hành của Luật 30/2023/QH15, gồm (1) ngày có hiệu lực và (2) các văn bản bị sửa đổi, bổ sung hoặc hết hiệu lực kể từ thời điểm đó. Khi truy hồi, hãy ưu tiên đoạn có cụm 'Luật này có hiệu lực thi hành từ ngày...' và đoạn liệt kê các luật/nghị quyết hết hiệu lực hoặc bị thay thế. Nếu có ngoại lệ hoặc điều khoản chuyển tiếp, cần giữ nguyên câu chữ theo quy định. Tránh suy đoán; chỉ cần trích đúng phần hiệu lực và ảnh hưởng đến văn bản khác."
}}


Ví dụ 3
Câu hỏi: "chất ma túy có bị cấm khi lái xe không?"
JSON:
{{
  "queries": [
    "hành vi bị nghiêm cấm chất ma túy khi điều khiển phương tiện",
    "lái xe có chất ma túy trong cơ thể có bị cấm theo quy định giao thông không",
    "điều khiển phương tiện trong cơ thể có chất ma túy hoặc chất kích thích bị cấm"
  ],
  "hyde_doc": "Câu hỏi hỏi về việc điều khiển phương tiện khi trong cơ thể có chất ma túy có bị cấm hay không. Cần truy hồi nhóm quy định 'các hành vi bị nghiêm cấm' trong văn bản về trật tự an toàn giao thông/luật giao thông. Đoạn cần lấy thường nêu rõ: điều khiển phương tiện khi trong cơ thể có chất ma túy hoặc chất kích thích khác mà pháp luật cấm sử dụng là hành vi bị nghiêm cấm. Nếu câu hỏi mở rộng sang mức phạt, khi đó mới tra thêm nghị định xử phạt tương ứng. Trọng tâm câu trả lời: khẳng định tính chất bị cấm theo quy định."
}}


Ví dụ 4
Câu hỏi: "Người có Giấy phép lái xe hạng D2 được điều khiển những loại xe nào, cụ thể đối với xe ô tô chở người?"
JSON:
{{
  "queries": [
    "Giấy phép lái xe hạng D2 được điều khiển xe ô tô chở người bao nhiêu chỗ",
    "bằng lái D2 lái được xe buýt và ô tô chở người từ trên 16 đến 29 chỗ",
    "hạng D2 xe ô tô chở người (kể cả xe buýt) trên 16 chỗ đến 29 chỗ kéo rơ moóc 750 kg"
  ],
  "hyde_doc": "Câu hỏi yêu cầu xác định phạm vi điều khiển của giấy phép lái xe hạng D2, trọng tâm là loại xe ô tô chở người và giới hạn số chỗ ngồi. Khi tra cứu trong bộ đề hoặc quy định phân hạng GPLX, cần tìm đoạn mô tả 'hạng D2' và các loại xe tương ứng. Nội dung thường nêu rõ xe ô tô chở người (kể cả xe buýt) có số chỗ trong một khoảng nhất định và có thể kèm điều kiện kéo rơ moóc đến một mức khối lượng thiết kế. Trả lời tốt nhất là trích nguyên câu mô tả hạng D2, đặc biệt phần 'trên 16 chỗ đến 29 chỗ' và điều kiện rơ moóc nếu có. Không suy diễn sang hạng khác nếu câu hỏi chỉ hỏi D2."
}}


Ví dụ 5
Câu hỏi: "Người lái xe có được phép vượt xe trên cầu hẹp có một làn đường không?"
JSON:
{{
  "queries": [
    "được phép vượt xe trên cầu hẹp có một làn đường không",
    "quy định vượt xe trên cầu hẹp một làn đường cong tầm nhìn hạn chế",
    "câu hỏi vượt xe trên cầu hẹp có một làn đường đáp án không được phép vượt"
  ],
  "hyde_doc": "Câu hỏi thuộc nhóm quy tắc vượt xe trong bộ đề giao thông, nhấn mạnh tình huống cầu hẹp chỉ có một làn và khu vực tầm nhìn hạn chế. Khi truy hồi, nên tìm câu/đoạn có cụm 'không được phép vượt' gắn với điều kiện 'cầu hẹp có một làn đường' hoặc 'đường cong tầm nhìn bị hạn chế'. Câu trả lời thường rất ngắn và dứt khoát theo dạng trắc nghiệm. Tránh thêm điều kiện ngoại lệ nếu trong đề không có. Ưu tiên trích đúng đáp án chuẩn của câu hỏi."
}}


Ví dụ 6
Câu hỏi: "Hiểu đúng về thiết bị an toàn cho trẻ em trên xe ô tô nhằm phòng ngừa chấn thương là như thế nào?"
JSON:
{{
  "queries": [
    "thiết bị an toàn cho trẻ em trên xe ô tô được hiểu như thế nào là đúng",
    "định nghĩa thiết bị an toàn trẻ em hạn chế di chuyển cơ thể giảm nguy cơ chấn thương khi va chạm",
    "thiết bị an toàn trẻ em tư thế ngồi hoặc nằm giảm chấn thương khi va chạm hoặc giảm tốc đột ngột"
  ],
  "hyde_doc": "Câu hỏi yêu cầu định nghĩa đúng về thiết bị an toàn cho trẻ em trên xe ô tô, mục tiêu là phòng ngừa chấn thương khi xảy ra va chạm hoặc giảm tốc đột ngột. Khi truy hồi trong bộ đề, cần tìm câu có cụm 'thiết bị an toàn cho trẻ em' và phần mô tả chức năng 'hạn chế sự di chuyển của cơ thể trẻ em'. Đáp án đúng thường nhấn mạnh trẻ em ở tư thế ngồi hoặc nằm trên xe và thiết kế nhằm giảm nguy cơ chấn thương trong tình huống va chạm. Không chọn mô tả sai như tư thế đứng hoặc chỉ áp dụng khi xe lưu thông bình thường. Cần trích nguyên mô tả đầy đủ của đáp án đúng."
}}


========================
BÂY GIỜ, HÃY THỰC HIỆN CHO CÂU HỎI SAU (không giải thích, chỉ JSON):

Câu hỏi: {question}
""".strip()

    resp = openai_call_once(
        model=model,
        input=[{"role": "user", "content": prompt}],
        text_format=MultiQueriesOut,
        store=False,  # không lưu logs nếu bạn muốn
    )

    try:
        data: Optional[MultiQueriesOut] = resp.output_parsed
        raw_queries = data.queries if data else []
        hyde_doc = (data.hyde_doc if data else "") or ""
        queries = [q.strip() for q in raw_queries if q and q.strip()]
    except Exception:
        queries, hyde_doc = [], ""

    # unique + luôn giữ câu gốc ở đầu, đủ k phần tử
    out, seen = [], set()
    for q in [question] + queries:
        key = q.lower()
        if key in seen:
            continue
        seen.add(key)
        out.append(q)
        if len(out) >= k:
            break

    if not out:
        out = [question]

    return out, hyde_doc


# 5. Reciprocal Rank Fusion

In [19]:
def doc_key(d):
    return (
        d.metadata.get("source"),
        d.metadata.get("page"),
        d.metadata.get("chunk_id"),
        (d.page_content or "")[:120],
    )

def rrf_fuse(ranked_lists: list[list], rrf_k: int = 60) -> list:
    scores = defaultdict(float)
    first_doc = {}

    for lst in ranked_lists:
        for rank, d in enumerate(lst, start=1):
            key = doc_key(d)
            if key not in first_doc:
                first_doc[key] = d
            scores[key] += 1.0 / (rrf_k + rank)

    fused = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [first_doc[k] for k, _ in fused]

def retrieve_hybrid_with_multiqueries(
    question: str,
    k_queries: int = 3,
    *,
    use_hyde: bool = True,
):
    queries, hyde_doc = generate_multi_queries_openai(question=question, k=k_queries)

    ranked_lists = [r_invoke(hybrid_retriever, q) for q in queries]

    # HyDE: chạy dense-only ensemble (vector)
    if use_hyde and hyde_doc and hyde_doc.strip():
        ranked_lists.append(r_invoke(hybrid_retriever, hyde_doc))

    return queries, ranked_lists, hyde_doc

# 6. Maximal Marginal Relevance

In [20]:
def mmr_select_docs(
    *,
    question: str,
    docs: list,
    embeddings,                 # HuggingFaceEmbeddings
    k: int = 30,
    lambda_mult: float = 0.5,
) -> list:
    if not docs:
        return []

    # HuggingFaceEmbeddings có embed_query + embed_documents :contentReference[oaicite:2]{index=2}
    q_emb = np.array(embeddings.embed_query(question), dtype=np.float32)

    doc_texts = [d.page_content for d in docs]
    doc_embs = embeddings.embed_documents(doc_texts)
    doc_embs = [np.array(e, dtype=np.float32) for e in doc_embs]

    k = min(k, len(docs))
    idxs = maximal_marginal_relevance(
        query_embedding=q_emb,
        embedding_list=doc_embs,
        lambda_mult=lambda_mult,
        k=k,
    )

    return [docs[i] for i in idxs]

# 7. Reranker

In [21]:
ce_model = HuggingFaceCrossEncoder(model_name="AITeamVN/Vietnamese_Reranker")
compressor = CrossEncoderReranker(model=ce_model, top_n=8)

def dedup_docs(docs):
    uniq, seen = [], set()
    for d in docs:
        key = (
            d.metadata.get("source"),
            d.metadata.get("page"),
            d.metadata.get("chunk_id"),
            d.page_content[:120],
        )
        if key in seen:
            continue
        seen.add(key)
        uniq.append(d)
    return uniq

def retrieve_image_text_then_rerank(
    question: str,
    question_id: Optional[str],
    k_queries: int = 3,
    max_candidates: int = 60,
    rerank_top_n: int = 10,

    # NEW knobs
    rrf_k: int = 60,
    mmr_k: int = 30,
    mmr_lambda: float = 0.5,
):
    # 1) qid BM25 (coi như 1 ranked list)
    qid_docs = bm25_by_qid(question_id, k=8) if question_id else []

    # 2) hybrid multiquery (n ranked lists)
    queries, hybrid_lists, _hyde_doc = retrieve_hybrid_with_multiqueries(question, k_queries=k_queries)

    # 3) RRF fuse
    all_lists = ([qid_docs] if qid_docs else []) + hybrid_lists
    fused_docs = rrf_fuse(all_lists, rrf_k=rrf_k)

    # 4) dedup + cap
    candidates = dedup_docs(fused_docs)[:max_candidates]

    # 5) MMR (dùng đúng text_embedder của bạn)
    if mmr_k and mmr_k > 0 and len(candidates) > 1:
        candidates = mmr_select_docs(
            question=question,
            docs=candidates,
            embeddings=text_embedder,
            k=mmr_k,
            lambda_mult=mmr_lambda,
        )

    # 6) cross-encoder rerank
    compressor.top_n = rerank_top_n
    reranked_docs = compressor.compress_documents(candidates, query=question)

    return {
        "question": question,
        "question_id": question_id,
        "queries": queries,
        "qid_docs": qid_docs,
        "rrf_fused": fused_docs,
        "candidates": candidates,
        "docs": reranked_docs,
    }

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

# 8. LLM response

In [22]:
def pil_to_data_url(img: Image.Image, fmt: str = "PNG") -> str:
    buf = io.BytesIO()
    img.save(buf, format=fmt)
    b64 = base64.b64encode(buf.getvalue()).decode("utf-8")
    return f"data:image/{fmt.lower()};base64,{b64}"

In [23]:
def image_to_note(user_pil_image: Image.Image, model="gpt-4o-mini") -> str:
    """1 call nhỏ để rút gọn ảnh thành ghi chú text."""
    resp = client.responses.create(
        model=model,
        input=[{
            "role": "user",
            "content": [
                {"type": "input_text", "text": "Tóm tắt nội dung ảnh trong 1–2 câu, chỉ mô tả những gì thấy được."},
                {"type": "input_image", "image_url": pil_to_data_url(user_pil_image)}
            ]
        }],
        store=False,
    )
    return (resp.output_text or "").strip()

In [24]:
def build_context(docs, max_chars: int = 12000):
    blocks, used = [], 0
    for i, d in enumerate(docs, 1):
        meta = d.metadata or {}
        header = f"[{i}] Source={meta.get('source')} | Page={meta.get('page')} | Chunk={meta.get('chunk_id')}\n"
        block = header + d.page_content.strip() + "\n\n"
        if used + len(block) > max_chars:
            break
        blocks.append(block)
        used += len(block)
    return "".join(blocks)

def has_qid_in_docs(docs, qid: str) -> bool:
    if not qid:
        return False
    needle = f"id: {qid}"
    return any(needle in (d.page_content or "") for d in docs)

MEMORY = []

def answer_openai_with_image_memory(
    question: str,
    docs,
    user_pil_image: Image.Image | None = None,
    model: str = "gpt-4o-mini",
    max_turns: int = 3,
):
    context = build_context(docs)

    system_msg = {
  "role": "system",
  "content": ("""
Bạn là trợ lý trả lời câu hỏi ôn thi Luật/Quy định giao thông đường bộ Việt Nam.

NGUỒN DUY NHẤT:
- Chỉ dùng thông tin trong phần "TÀI LIỆU" (và "ẢNH" nếu có).
- Không dùng kiến thức bên ngoài. Không bịa điều/khoản/mức phạt.

CHUẨN HÓA TÀI LIỆU (BẮT BUỘC, ÂM THẦM):
- Nếu văn bản trong TÀI LIỆU bị lỗi OCR/poor grammar (ví dụ: "ch ủ"→"chủ", "B ộ"→"Bộ", "ho ặc"→"hoặc", xuống dòng/khuyết dấu/ thừa khoảng trắng), hãy tự sửa cho đúng chính tả và định dạng tiếng Việt.
- Chỉ sửa lỗi kỹ thuật (chính tả, dấu, khoảng trắng, ký tự bị tách, xuống dòng), TUYỆT ĐỐI không thêm/bớt ý hoặc đổi nghĩa pháp lý.

ĐẦU RA (BẮT BUỘC):
- Chỉ 1 đoạn text duy nhất (1 paragraph), ngắn gọn, đi thẳng vào đáp án.
- Không bullet, không liệt kê, không giải thích lan man.

VĂN PHONG:
- Nếu câu hỏi dạng Có/Không: bắt đầu bằng "Có," hoặc "Không," rồi viết 1 câu khẳng định theo đúng nội dung tài liệu.
- Nếu trong tài liệu có cụm “bị nghiêm cấm/được phép/không được phép”: phải dùng đúng cụm đó trong câu trả lời.
- Ưu tiên dùng câu chữ trong tài liệu, nhưng áp dụng quy tắc CHUẨN HÓA ở trên để câu văn mạch lạc.
- Nếu không đủ dữ liệu để kết luận: chỉ trả lời đúng 1 câu "Chưa đủ thông tin trong tài liệu".

XỬ LÝ ẢNH:
- Nếu có ảnh: dùng ảnh để nhận diện đối tượng/tình huống rồi kết luận theo TÀI LIỆU.
- Nếu ảnh và tài liệu mâu thuẫn: ưu tiên TÀI LIỆU.

VÍ DỤ:
Q: chất ma túy có bị cấm khi lái xe không?
A: Có, điều khiển phương tiện tham gia giao thông đường bộ mà trong cơ thể có chất ma túy hoặc chất kích thích khác mà pháp luật cấm sử dụng là hành vi bị nghiêm cấm.
""").strip()
}


    image_note = ""
    if user_pil_image is not None:
        image_note = image_to_note(user_pil_image, model=model)

    user_msg = {
        "role": "user",
        "content": (
            f"TÀI LIỆU:\n{context}\n\n"
            + (f"ẢNH (mô tả): {image_note}\n\n" if image_note else "")
            + f"CÂU HỎI:\n{question}"
        )
    }

    input_msgs = [system_msg] + MEMORY + [user_msg]

    resp = client.responses.create(model=model, input=input_msgs, store=False)

    # output_text: OK nếu SDK mới; nếu lỗi thì bạn báo mình để mình đưa fallback đúng version bạn đang dùng
    answer = (getattr(resp, "output_text", "") or "").strip()

    # cập nhật memory
    if image_note:
        MEMORY.append({"role": "assistant", "content": f"[GHI CHÚ ẢNH]: {image_note}"})
    MEMORY.append({"role": "user", "content": question})
    MEMORY.append({"role": "assistant", "content": answer})

    keep = max_turns * 2
    if len(MEMORY) > keep:
        del MEMORY[:-keep]

    return answer

# 9. Full Pipeline

In [40]:
def _distance_to_similarity_cosine(distance: float) -> float:
    # mapping kiểu "thực dụng": similarity ~= 1 - distance
    return max(0.0, 1.0 - float(distance))


def _is_image_match_cosine_distance(
    distances: List[float],
    *,
    min_img_similarity: float,
    margin: float,
) -> bool:
    """
    Chroma trả distance: nhỏ -> giống.
    Với cosine distance, hay dùng similarity ~= 1 - distance.
    Accept nếu:
      - top1 đủ giống (distance <= 1 - min_similarity)
      - nếu có top2: top1 phải "tách bạch" đủ (d2 - d1 >= margin)
    """
    if not distances:
        return False

    d1 = float(distances[0])
    max_dist = 1.0 - float(min_img_similarity)
    if d1 > max_dist:
        return False

    if len(distances) >= 2:
        d2 = float(distances[1])
        if (d2 - d1) < float(margin):
            return False

    return True


def _tap(debug: bool, fn):
    """Runnable 'tap' để in debug mà không phá pipeline."""
    def _inner(x):
        if debug:
            fn(x)
        return x
    return RunnableLambda(_inner)


# =========================================================
# RRF + MMR helpers
# =========================================================

def _doc_text(doc: Any) -> str:
    return getattr(doc, "page_content", None) or str(doc) or ""


def _doc_key(doc: Any) -> str:
    """
    Key ổn định để gộp doc (dùng cho RRF). Ưu tiên metadata nếu có.
    Fallback: hash theo text.
    """
    meta = getattr(doc, "metadata", None) or {}
    src = meta.get("source")
    page = meta.get("page")
    chunk = meta.get("chunk_id") or meta.get("chunk")
    if src is not None or page is not None or chunk is not None:
        return f"{src}|{page}|{chunk}"
    # fallback theo text
    return f"text:{hash(_doc_text(doc))}"


def rrf_fuse(ranked_lists: List[List[Any]], *, rrf_k: int = 60) -> List[Any]:
    """
    Reciprocal Rank Fusion.
    score(d) = sum_i 1 / (rrf_k + rank_i(d))
    """
    scores: Dict[str, float] = {}
    first_obj: Dict[str, Any] = {}

    for lst in ranked_lists:
        if not lst:
            continue
        for rank, doc in enumerate(lst, start=1):
            key = _doc_key(doc)
            if key not in first_obj:
                first_obj[key] = doc
            scores[key] = scores.get(key, 0.0) + 1.0 / (float(rrf_k) + float(rank))

    # sort by score desc
    items = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
    return [first_obj[k] for k, _ in items]


def _to_vec(x: Any) -> List[float]:
    # đảm bảo vector là list[float]
    if x is None:
        return []
    if isinstance(x, list):
        return [float(v) for v in x]
    # numpy array / torch tensor -> list
    try:
        return [float(v) for v in list(x)]
    except Exception:
        return []


def _cosine_sim(a: List[float], b: List[float]) -> float:
    if not a or not b or len(a) != len(b):
        return 0.0
    dot = 0.0
    na = 0.0
    nb = 0.0
    for i in range(len(a)):
        ai = float(a[i])
        bi = float(b[i])
        dot += ai * bi
        na += ai * ai
        nb += bi * bi
    denom = math.sqrt(na) * math.sqrt(nb)
    return dot / denom if denom > 0 else 0.0


def _embed_query(embeddings: Any, text: str) -> List[float]:
    # hỗ trợ LangChain Embeddings hoặc callable
    if embeddings is None:
        return []
    if hasattr(embeddings, "embed_query"):
        return _to_vec(embeddings.embed_query(text))
    if callable(embeddings):
        return _to_vec(embeddings(text))
    return []


def _embed_docs(embeddings: Any, texts: List[str]) -> List[List[float]]:
    if embeddings is None:
        return [[] for _ in texts]
    if hasattr(embeddings, "embed_documents"):
        return [_to_vec(v) for v in embeddings.embed_documents(texts)]
    if callable(embeddings):
        return [_to_vec(embeddings(t)) for t in texts]
    return [[] for _ in texts]


def mmr_select_docs(
    *,
    question: str,
    docs: List[Any],
    embeddings: Any,
    k: int = 30,
    lambda_mult: float = 0.5,
) -> List[Any]:
    """
    MMR (Maximal Marginal Relevance) selection.
    - ưu tiên doc liên quan query
    - đồng thời giảm trùng lặp giữa các doc đã chọn
    """
    if not docs:
        return []
    k = min(int(k), len(docs))
    lam = float(lambda_mult)

    texts = [_doc_text(d) for d in docs]
    qvec = _embed_query(embeddings, question)
    dvecs = _embed_docs(embeddings, texts)

    # precompute sim(query, doc)
    sim_q = [_cosine_sim(qvec, dv) for dv in dvecs]

    selected: List[int] = []
    remaining = set(range(len(docs)))

    for _ in range(k):
        best_i = None
        best_score = -1e9

        for i in remaining:
            if not selected:
                score = sim_q[i]
            else:
                # max similarity to already selected
                max_sim_to_sel = max(_cosine_sim(dvecs[i], dvecs[j]) for j in selected)
                score = lam * sim_q[i] - (1.0 - lam) * max_sim_to_sel

            if score > best_score:
                best_score = score
                best_i = i

        if best_i is None:
            break
        selected.append(best_i)
        remaining.remove(best_i)

    return [docs[i] for i in selected]


# =========================================================
# Build LCEL chain
# =========================================================

def build_rag_multimodal_chain(
    *,
    # deps bạn đã có sẵn trong môi trường
    chroma_img_collection,            # vd: chroma_img600._collection
    hybrid_retriever,                 # vd: hybrid_retriever
    compressor,                       # vd: cross-encoder compressor

    # functions bạn đã build sẵn
    embed_uploaded_bytes_to_vec,
    bm25_by_qid,
    generate_multi_queries_openai,
    r_invoke,
    dedup_docs,
    has_qid_in_docs,
    answer_openai_with_image_memory,

    # config
    n_img_results: int = 5,
    min_img_similarity: float = 0.95,
    margin: float = 0.08,
    k_queries: int = 3,
    max_candidates: int = 60,

    # RRF/MMR config
    rrf_k: int = 60,
    per_query_k: int = 20,
    use_mmr: bool = True,
    mmr_k: int = 30,
    mmr_lambda: float = 0.5,
    mmr_embeddings=None,              # mặc định lấy global text_embedder nếu có

    # rerank/llm
    rerank_top_n: int = 10,
    model: str = "gpt-4o-mini",
    debug: bool = True,
):
    if mmr_embeddings is None:
        mmr_embeddings = globals().get("text_embedder", None)

    # ---------- Step 1: image -> question_id ----------
    def step_image(d: Dict[str, Any]) -> Dict[str, Any]:
        img_bytes: Optional[bytes] = d.get("img_bytes")
        if img_bytes is None:
            return {**d, "user_pil": None, "question_id": None, "img_debug": {}}

        user_pil = Image.open(io.BytesIO(img_bytes)).convert("RGB")
        qvec = embed_uploaded_bytes_to_vec(img_bytes)

        res = chroma_img_collection.query(
            query_embeddings=[qvec],
            n_results=n_img_results,
            include=["distances", "metadatas"],
        )

        dists = (res.get("distances") or [[]])[0]
        metas = (res.get("metadatas") or [[]])[0]

        best_qid = metas[0].get("question_id") if metas else None
        matched = _is_image_match_cosine_distance(
            dists, min_img_similarity=min_img_similarity, margin=margin
        )
        question_id = best_qid if matched else None

        sim1 = _distance_to_similarity_cosine(dists[0]) if dists else None
        sim2 = _distance_to_similarity_cosine(dists[1]) if len(dists) >= 2 else None

        img_debug = {
            "best_qid": best_qid,
            "matched": matched,
            "distances": dists,
            "similarities": [sim1, sim2],
            "top_metadatas": metas,
        }

        return {**d, "user_pil": user_pil, "question_id": question_id, "img_debug": img_debug}

    # ---------- Step 2: generate multi-queries ----------
    def step_multiqueries(d: Dict[str, Any]) -> Dict[str, Any]:
        question = d["question"]
        queries, hyde_doc = generate_multi_queries_openai(question, k=k_queries, model=model)
        return {**d, "queries": queries, "hyde_doc": hyde_doc}

    # ---------- Step 3: retrieve ranked lists (qid BM25 + hybrid per query) ----------
    def get_qid_docs(d: Dict[str, Any]) -> List[Any]:
        qid = d.get("question_id")
        return bm25_by_qid(qid, k=8) if qid else []

    def get_hybrid_ranked_lists(d: Dict[str, Any]) -> List[List[Any]]:
        ranked_lists: List[List[Any]] = []

        for q in d.get("queries") or []:
            lst = r_invoke(hybrid_retriever, q) or []
            ranked_lists.append(lst[:per_query_k])

        hyde_doc = (d.get("hyde_doc") or "").strip()
        if hyde_doc:
            lst = r_invoke(hybrid_retriever, hyde_doc) or []
            ranked_lists.append(lst[:per_query_k])

        return ranked_lists

    def merge_candidates(d: Dict[str, Any]) -> Dict[str, Any]:
        qid_docs = d.get("qid_docs") or []
        hybrid_ranked_lists = d.get("hybrid_ranked_lists") or []

        fused = rrf_fuse([qid_docs] + hybrid_ranked_lists, rrf_k=rrf_k)
        candidates = dedup_docs(fused)[:max_candidates]

        return {**d, "candidates": candidates}

    # ---------- Step 3.5: MMR (optional) ----------
    def step_mmr(d: Dict[str, Any]) -> Dict[str, Any]:
        if (not use_mmr) or (mmr_embeddings is None):
            return d
        candidates = d.get("candidates") or []
        mmr_docs = mmr_select_docs(
            question=d["question"],
            docs=candidates,
            embeddings=mmr_embeddings,
            k=mmr_k,
            lambda_mult=mmr_lambda,
        )
        return {**d, "candidates": mmr_docs, "candidates_mmr": mmr_docs}

    # ---------- Step 4: rerank ----------
    def step_rerank(d: Dict[str, Any]) -> Dict[str, Any]:
        compressor.top_n = rerank_top_n
        reranked_docs = compressor.compress_documents(d["candidates"], query=d["question"])
        return {**d, "docs": reranked_docs}

    # ---------- Step 5: attach_img ----------
    def step_attach_img(d: Dict[str, Any]) -> Dict[str, Any]:
        qid = d.get("question_id")
        docs = d.get("docs") or []
        attach_img = bool(qid) or (qid is not None and has_qid_in_docs(docs, qid))
        return {**d, "attach_img": attach_img}

    # ---------- Step 6: answer ----------
    def step_answer(d: Dict[str, Any]) -> Dict[str, Any]:
        answer = answer_openai_with_image_memory(
            question=d["question"],
            docs=d["docs"],
            user_pil_image=(d["user_pil"] if d["attach_img"] else None),
            model=model,
            max_turns=3,
        )
        return {**d, "answer": answer}

    # =========================================================
    # LCEL chain (dict in -> dict out)
    # =========================================================
    chain = (
        RunnablePassthrough()
        | RunnableLambda(step_image)
        | RunnableLambda(step_multiqueries)
        | RunnablePassthrough.assign(
            qid_docs=RunnableLambda(get_qid_docs),
            hybrid_ranked_lists=RunnableLambda(get_hybrid_ranked_lists),
        )
        | RunnableLambda(merge_candidates)
        | RunnableLambda(step_mmr)
        | RunnableLambda(step_rerank)
        | RunnableLambda(step_attach_img)
        | RunnableLambda(step_answer)
    )

    return chain


# =========================================================
# Wrapper function giống run_rag_multimodal cũ (invoke LCEL)
# =========================================================

def run_rag_multimodal_lcel(
    question: str,
    img_bytes: bytes | None = None,
    *,
    chain,
    debug: bool = True,
):
    out = chain.invoke({"question": question, "img_bytes": img_bytes})

    return {
        "question": out.get("question"),
        "question_id": out.get("question_id"),
        "img_debug": out.get("img_debug"),
        "queries": out.get("queries"),
        "hyde_doc": out.get("hyde_doc"),
        "qid_docs": out.get("qid_docs"),
        "candidates": out.get("candidates"),
        "docs": out.get("docs"),
        "attach_img": out.get("attach_img"),
        "answer": out.get("answer"),
    }

In [41]:
rag_chain = build_rag_multimodal_chain(
    chroma_img_collection=chroma_img600._collection,
    hybrid_retriever=hybrid_retriever,
    compressor=compressor,

    embed_uploaded_bytes_to_vec=embed_uploaded_bytes_to_vec,
    bm25_by_qid=bm25_by_qid,
    generate_multi_queries_openai=generate_multi_queries_openai,
    r_invoke=r_invoke,
    dedup_docs=dedup_docs,
    has_qid_in_docs=has_qid_in_docs,
    answer_openai_with_image_memory=answer_openai_with_image_memory,

    # config
    min_img_similarity=0.95,
    margin=0.08,
    k_queries=3,
    max_candidates=60,
    rerank_top_n=10,
    model="gpt-4o-mini",
    debug=True,
)

# 10. End to End demo

In [43]:
def print_docs(title: str, docs: list, top_n: int = 5):
    print("\n" + "=" * 90)
    print(title)
    print("=" * 90)
    for i, d in enumerate(docs[:top_n], start=1):
        meta = getattr(d, "metadata", {}) or {}
        src = meta.get("source")
        page = meta.get("page")
        chunk = meta.get("chunk_id")
        text = (getattr(d, "page_content", "") or "").replace("\n", " ").strip()
        snippet = (text[:240] + "…") if len(text) > 240 else text
        print(f"[{i}] source={src} | page={page} | chunk={chunk}")
        print(f"    {snippet}")

# (Tuỳ chọn) reset memory hội thoại để demo "clean"
try:
    MEMORY.clear()
except Exception:
    pass

question = "Độ tuổi nào cho phép người tham gia giao thông điều khiển phương tiện"
out = run_rag_multimodal_lcel(question, img_bytes=None, chain=rag_chain, debug=True)

print("\n🧠 HyDE doc:\n", out.get("hyde_doc"))
print("\n✅ ANSWER:\n", out["answer"])
print("\n🆔 question_id:", out["question_id"])
print("\n🔎 queries:")
for i, q in enumerate(out["queries"] or [], 1):
    print(f"  {i}. {q}")

print_docs("📌 Top reranked docs (docs)", out["docs"] or [], top_n=8)



🧠 HyDE doc:
 Câu hỏi yêu cầu xác định độ tuổi hợp lệ để tham gia giao thông và điều khiển phương tiện tại Việt Nam. Cần tìm thông tin về quy định độ tuổi tối thiểu trong luật giao thông đường bộ, thường có một số độ tuổi cụ thể như 18 tuổi đối với ô tô và 16 tuổi đối với xe mô tô, cùng với những điều kiện bổ sung liên quan đến loại phương tiện. Cần lưu ý rằng mỗi hạng GPLX có thể có yêu cầu độ tuổi khác nhau, vì vậy thông tin về mỗi hạng cần được nêu rõ, ví dụ như độ tuổi cấp GPLX cho xe mô tô, ô tô hạng B1, B2, và C.

✅ ANSWER:
 Người đủ 16 tuổi trở lên được điều khiển xe gắn máy; đủ 18 tuổi trở lên được cấp giấy phép lái xe hạng A1, A, B1, B, C1; đủ 21 tuổi trở lên được cấp giấy phép lái xe hạng C, BE; đủ 24 tuổi trở lên được cấp giấy phép lái xe hạng D1, D2, C1E, CE; đủ 27 tuổi trở lên được cấp giấy phép lái xe hạng D, D1E, D2E, DE. Tuổi tối đa để lái xe ô tô chở người trên 29 chỗ là đủ 57 tuổi đối với nam và đủ 55 tuổi đối với nữ.

🆔 question_id: None

🔎 queries:
  1. Độ tuổi nào 

In [39]:
from pathlib import Path

img_path = Path("/content/img_question_227.png")
img_bytes = img_path.read_bytes()

question = "Biển báo trong ảnh này yêu cầu người lái xe làm gì?"
out = run_rag_multimodal_lcel(question, img_bytes=img_bytes, chain=rag_chain, debug=True)

print("\n✅ ANSWER:\n", out["answer"])
print("\n🖼️ img_debug:", out["img_debug"])
print_docs("📌 Top reranked docs (docs)", out["docs"] or [], top_n=8)



🖼️ Image retrieve (top 5) | matched=True | min_sim=0.95 | margin=0.08
[01] qid=Q227 | dist=0.000000 | sim=1.0000 | source=None | page=None
[02] qid=Q288 | dist=0.337293 | sim=0.6627 | source=None | page=None
[03] qid=Q287 | dist=0.440275 | sim=0.5597 | source=None | page=None
[04] qid=Q229 | dist=0.493907 | sim=0.5061 | source=None | page=None
[05] qid=Q425 | dist=0.495488 | sim=0.5045 | source=None | page=None
------------------------------------------------------------------------------------------
Top metadatas (raw):
[{'id': 'Q227_img_1',
  'image_path': '/content/drive/Othercomputers/My '
                'Laptop/Docs/multimodal-rag/data/images/qa600/img_question_227.png',
  'modality': 'text and image',
  'question_id': 'Q227',
  'source_doc': 'Bộ 600 câu hỏi dùng cho sát hạch lái xe cơ giới đường bộ.pdf'},
 {'id': 'Q288_img_1',
  'image_path': '/content/drive/Othercomputers/My '
                'Laptop/Docs/multimodal-rag/data/images/qa600/img_question_288.png',
  'modality': 't